Data from: https://www.pxweb.bfs.admin.ch/default.aspx

In [1]:
# imports
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib.pyplot import show
import json
import warnings
%matplotlib inline
sns.set_context('notebook')
pd.options.mode.chained_assignment = None  # default='warn'
pd.set_option('display.max_columns', 500) # show more columns
nan = np.nan # store numpy.nan in 'nan'

In [2]:
raw_data= pd.read_csv('raw_data/deaths_over40_utf8_raw.csv', sep=';', header=1)

# rename the columns
d = {'Jahr':'year', 'Kanton (-) / Bezirk (>>) / Gemeinde (......)': 'commune_name', 
     'Zivilstand - Total Unter 20 Jahre':'death age 0-20',
     'Zivilstand - Total 20-29 Jahre':'death age 20-29',
     'Zivilstand - Total 30-39 Jahre':'death age 30-39',
                   'Zivilstand - Total 40-49 Jahre':'death age 40-49',
                   'Zivilstand - Total 50-59 Jahre':'death age 50-59',
                    'Zivilstand - Total 60-69 Jahre':'death age 60-69',
                    'Zivilstand - Total 70-79 Jahre':'death age 70-79',
                    'Zivilstand - Total 80-89 Jahre':'death age 80-89',
                    'Zivilstand - Total 80 Jahre und mehr':'death age 90+',
    'Zivilstand - Total Altersklasse - Total':'total deaths'}
raw_data.rename(columns=d, inplace=True)
raw_data15 = raw_data[raw_data['year'] == 2015]


In [3]:
data = pd.DataFrame(raw_data15['commune_name'])
data.head()

,commune_name
4998,Schweiz / Suisse / Svizzera / Switzerland
4999,- Zürich
5000,>> Bezirk Affoltern
5001,......0001 Aeugst am Albis
5002,......0002 Affoltern am Albis


In [17]:
comune_canton_dict = {}
curr_canton = None
idx=1
canton_idx=0
for index, row in data.iterrows():
    cn = row['commune_name']
    if '>>' in cn:
        continue # this is a bezirk
    elif '......' in cn:
        # this is a commune
        cn = cn.replace('......', '')
        cn_id = int(cn.split()[0])
        cn = ' '.join(cn.split()[1:])
        comune_canton_dict[idx] = [cn_id, cn, curr_canton, canton_idx]
        idx+= 1
    elif '- ' in cn:
        # this is a canton
        cn = cn.replace('- ', '')
        curr_canton = cn
        canton_idx +=1
        print('new canton', cn)
    else:
        warnings.warn('unknown name: '+cn)


/Users/baptisteraemy/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:23: UserWarning: unknown name: Schweiz / Suisse / Svizzera / Switzerland


new canton Zürich
new canton Bern / Berne
new canton Luzern
new canton Uri
new canton Schwyz
new canton Obwalden
new canton Nidwalden
new canton Glarus
new canton Zug
new canton Fribourg / Freiburg
new canton Solothurn
new canton Basel-Stadt
new canton Basel-Landschaft
new canton Schaffhausen
new canton Appenzell Ausserrhoden
new canton Appenzell Innerrhoden
new canton St. Gallen
new canton Graubünden / Grigioni / Grischun
new canton Aargau
new canton Thurgau
new canton Ticino
new canton Vaud
new canton Valais / Wallis
new canton Neuchâtel
new canton Genève
new canton Jura


In [19]:
d = {'aaare': ['Bern']}
commune_canton = pd.DataFrame.from_dict(comune_canton_dict, orient='index')
commune_canton.rename(columns={commune_canton.columns[0]:'commune_id',
                               commune_canton.columns[1]:'commune_name', 
                               commune_canton.columns[2]:'canton',
                               commune_canton.columns[3]:'canton_id'}, inplace=True)
commune_canton = commune_canton.sort_values('commune_id')

In [20]:
commune_canton.head()

,commune_id,commune_name,canton,canton_id
1,1,Aeugst am Albis,Zürich,1
2,2,Affoltern am Albis,Zürich,1
3,3,Bonstetten,Zürich,1
4,4,Hausen am Albis,Zürich,1
5,5,Hedingen,Zürich,1


In [21]:
commune_canton.to_csv("../municipalities/2013/commune_canton_map.csv", index=False)
commune_canton.to_csv("../municipalities/2014/commune_canton_map.csv", index=False)
commune_canton.to_csv("../municipalities/2015/commune_canton_map.csv", index=False)
commune_canton.to_csv("../municipalities/2016/commune_canton_map.csv", index=False)